# SVR

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

## Memuat Data


In [2]:
X_train = pd.read_csv('dataset/X_train.csv')
X_test = pd.read_csv('dataset/X_test.csv')
y_train = pd.read_csv('dataset/y_train.csv').values.ravel()
y_test = pd.read_csv('dataset/y_test.csv').values.ravel()

## Pelatihan Model dengan Tuning


In [ ]:
svr_params = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1.0],
    'epsilon': [0.1, 0.2]
}
svr_grid = RandomizedSearchCV(
    SVR(),
    svr_params,
    n_iter=8,
    cv=3,
    scoring='r2'
)
svr_grid.fit(X_train, y_train)
svr_best = svr_grid.best_estimator_

print("\n=== SVR Best Parameters ===")
print(svr_grid.best_params_)

# tidak jadi, karena waktu training sangat lama

## Evaluasi Model


In [ ]:
y_train_pred = svr_best.predict(X_train)
y_test_pred = svr_best.predict(X_test)

metrics = {
    'Model': 'SVR',
    'Train MAE': mean_absolute_error(y_train, y_train_pred),
    'Test MAE': mean_absolute_error(y_test, y_test_pred),
    'Train RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred)),
    'Test RMSE': np.sqrt(mean_squared_error(y_test, y_test_pred)),
    'Train R2': r2_score(y_train, y_train_pred),
    'Test R2': r2_score(y_test, y_test_pred)
}

cv_scores = cross_val_score(svr_best, X_train, y_train, cv=5, scoring='r2')
metrics['CV R2 Mean'] = cv_scores.mean()
metrics['CV R2 Std'] = cv_scores.std()

## Ringkasan Hasil

In [ ]:
results_df = pd.DataFrame([metrics])
print("\n=== Hasil Evaluasi SVR ===")
print(results_df)

## Visualisasi Prediksi vs Aktual


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Exam Score')
plt.ylabel('Predicted Exam Score')
plt.title('SVR: Prediksi vs Aktual')
plt.tight_layout()
plt.show()

## Visualisasi Metrik


In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(x=['Train R2', 'Test R2', 'CV R2 Mean'], y=[metrics['Train R2'], metrics['Test R2'], metrics['CV R2 Mean']])
plt.title('SVR: Perbandingan R2 Score')
plt.ylabel('R2 Score')
plt.tight_layout()
plt.show()

## Menyimpan Model dan Hasil


In [ ]:
joblib.dump(svr_best, 'svr_model.pkl')
results_df.to_csv('svr_results.csv', index=False)
print("\nModel disimpan sebagai 'svr_model.pkl'")
print("Hasil evaluasi disimpan sebagai 'svr_results.csv'")